
---
# Processing passive seismic data: Retrieving earthquake data

This notebook has some practical processing activities of the Course **ProSeisSN**. It deals with data processing using a passive seismic dataset using [ObsPy](https://docs.obspy.org/).

#### Dependencies: Obspy, Numpy, Matplotlib

#### Reset the Jupyter/IPython notebook in order to run it again, press:
***Kernel*** -> ***Restart & Clear Output***

In [ ]:
# If needed for mathplotlib
# conda install -c conda-forge ipympl

In [ ]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
"""
Activate the ipympl backend by including
%matplotlib ipympl
magic in the notebook or by
%matplotlib widget
which will have the same effect.

Other magic:
%matplotlib inline
%matplotlib notebook
"""
%matplotlib widget

# Import specialized ObsPy packages + sanity.
try:
    import obspy
    print('obspy version==>', obspy.__version__)
except TypeError:
    print('Stopping RUNTIME. ObsPy not found.')
    exit()

from obspy import read
from obspy import UTCDateTime
from obspy import read, Stream
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.header import FDSNException

# Local routines. This is just a memo, does not do anything.
from utils import period2freq, freq2period
_ = freq2period(1.)

ModuleNotFoundError: No module named 'ipympl'

---
## Accessing a local data

This lecture serves as a basic introduction to using Obspy. Useful links are the official [Obspy Tutorial](https://docs.obspy.org/tutorial/index.html), and the [Seismo-Live Juypter Notebooks for Seismology](https://krischer.github.io/seismo_live_build/tree/index.html).

We work with data form an event occured in Acre in 2024, downloaded using [IRIS Wilber](http://www.iris.edu/wilber3).

|EventID | Time | Lat | Long | Depth(km) | Mww | Location |
| :- | :-: | :- | :- | :-: | :-: | :- |
|11793085|2024-01-20T21:31|-7.2879|-71.464|607|6.6|WESTERN BRAZIL|

We will use a data file in miniSEED format: [wb11793085_ir.mseed](https://github.com/jandyr/ProSeisSN_Nbk/blob/main/wb11793085_ir.mseed).

Choose stations with epicentral distances $\left[20^{\circ},\,70^{\circ}\right]$; waves travel in the laterally homogeneous Mantle, with P and S arrivals.

Of the 149 stations operational during the event, select station IU HKT, distant $44^{\circ}$ from the event.

|Station|Seismometer| Lat | Long | Depth(m) | Channels |
| :- | :- | :- | :- | :-: | :-: |
|HKT: Hockley1 Texas|Streckeisen STS-6A VBB |29.96|-95.84|93|BH1, BH2, BHZ|

**Phase Arrivals**

|Phase|$\Delta t$| TIme |
| :- | :-: | :- |
|P|+7m 14s|21:38:19|
|PP|+9m 9s |21:40:14|
|S|+13m 4s|21:44:09|
|SS|+16m 38s|21:47:43|

In [ ]:
# Specify event's start and end times
starttime = UTCDateTime("2024-01-20T21:37:19.019539Z") - 60
endtime = UTCDateTime("2024-01-20T21:58:18.994539Z")
print(starttime, endtime)

# Use wildcards to select all three HH* channels
net = "IU"
sta = "HKT"
loc = "00"
chan = "HH*"

# Specify client. Opt:from obspy.clients.earthworm import Client
# Get waveforms with instrument response into a stream
# List of ObsPy clients: clients = ["IRIS", "NCEDC", "USGS", "GEONET", "RESIF", "INGV", "BGR", "ODC", "SCEDC"]
try:
  client = Client("IRIS")
  st = client.get_waveforms(net, sta, loc, chan, starttime, endtime, attach_response = True)
except FDSNException:
  print(f"Chosen client is not working.")
  from obspy.clients.earthworm import Client
  client = Client("ETH")
  st = client.get_waveforms(net, sta, loc, chan, starttime, endtime, attach_response = True)

The first letter is 'y', 'Y', or 'n'.



- Plot all the 3-component/trace stream. The Vertical, z-component, is the last one.

In [ ]:
# Print stream information
print(st)
# Note the 3-component/trace stream. The Vertical, z-component, is the last.
#  Plot the 3 traces. Note the implicit for loop from 0 to 2 (=3-1!)
#  ├─+──> 1st horizontal coponent H1
#  │ ├──> vertical component HZ
st[0:3].plot();

---
### Remove the instrument response

_(i)  If necessary make a *deep* copy of the original stream as the function **remove_response** acts on the data, overwriting the original stream._

_(ii) Express the ground motion in velocity, **output = 'VEL'**, in units of velocity (m/s)._

The process involves multiplication with the inverse of the instrument response, which usually attain small numerical values at both frequency ends. This results in the signal+noise being multiplied by relatively large numbers, making noise contribution large. This can be alleviated by tapering the signal or imposing a *cap* to the inverse of the instrument response.

_Caution:_ If you run the code below more than once the function **remove_response** will corrupt the data because the original stream is lost. Use the copy of the original stream if needed.

In [2]:
# Deep copy of original stream
st0 = st.copy()
# Remove instrument response in all the 3 components/traces. Other options: output = 'DISP', 'ACC'
st.remove_response(output = 'VEL')
# Compare the Z components
st0[-1].plot();
st[-1].plot(color = 'blue');

NameError: name 'st' is not defined

### The frequency content

The [spectrogram](https://docs.obspy.org/tutorial/code_snippets/plotting_spectrograms.html) shows the power in frequency bands (along the y-axis) for every few samples in time (along the x-axis). Warmer colours indicate more power.

The station has most of it's power below 1 Hz, which is common for a time-period dominated by seismic noise.

In [ ]:
# Construct a figure for BHZ trace
fig, ax = plt.subplots()
_ = st[-1].spectrogram(log=True, dbscale=True, axes=ax, show=True, cmap = "jet")
plt.xlabel("Time(s)")
plt.ylabel("Frequency(Hz)")
#plt.title("Plot Title")
#plt.legend() # If you have multiple lines in the plot
plt.grid(True) # Add a grid to the plot
plt.show()